#Data Engineer Challenge - Miguel Guerrero

obs: Se importan las librerias dentro del segmento de codigo de cada solucion con el objetivo que sea utilizado unicamente ahi a pesar que puede ser reutilizado en otro segmento
esto con el objetivo de aportar claridad a las librerias necesarias para el funcionamiento de la solucion

In [ ]:
from memory_profiler import profile

# Las top 10 fechas donde hay más tweets. 
Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días. Debe incluir las siguientes funciones:
# Enfoque: Tiempo de ejecucion optimizado

Para este caso convertimos el archivo json a un dataframe de pandas por los beneficios que este tiene en la optimizacion de las operaciones con datos.
Como se esta priorizando el tiempo de ejecucion por sobre de la memoria.

In [11]:
import os
import ujson as json
import pandas as pd
from datetime import datetime
from typing import List, Tuple

#Se define la funcion que nos devolvera la lista de los 10 usuarios con mas tweets
def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    
    #Se nombra el archivo .parquet
    parquet_file = file_path.replace('.json', '.parquet')
    
    #Se verifica si el archivo .parquet existe, si existe se lee, si no se crea
    if os.path.exists(parquet_file):
        df = pd.read_parquet(parquet_file)
    else:
        df = pd.read_json(file_path, lines=True)
        df.to_parquet(parquet_file, index=False)
    

    #Se convierte la columna date a datetime de pandas y extraemos solo la fecha
    df['date'] = pd.to_datetime(df['date']).dt.date
    
    #Se extrae el nombre de usuario de la columna user
    df['username'] = df['user'].apply(lambda x: x.get('username') if isinstance(x, dict) else None)
    
    #Se elimina la columna user
    del df['user']

    #Se agrupa el DataFrame por fecha y nombre de usuario y se cuenta el número de tweets para cada combinación.
    grouped = df.groupby(['date', 'username']).size().reset_index(name='count')

    #Se obtiene las 10 fechas con el mayor número de tweets.
    top_dates_df = df['date'].value_counts().nlargest(10).reset_index()

    
    result = []
    for _, row in top_dates_df.iterrows():
        date = row['date']
        top_user_df = grouped[grouped['date'] == date].nlargest(1, 'count')
        top_user = top_user_df['username'].iloc[0]
        result.append((date, top_user))
    
    return result

q1_time('./farmers-protest-tweets-2021-2-4.json')


[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

# Enfoque: Memoria en uso optimizada

Para este se utiliza el archivo json y se lee linea por linea.


In [9]:
from datetime import datetime
from typing import List, Tuple
from collections import defaultdict
import ujson as json

def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    
    # Diccionario para mantener el conteo de tweets por fecha
    date_counts = defaultdict(int)
    # Diccionario para mantener el conteo de tweets de usuario por fecha
    user_date_counts = defaultdict(lambda: defaultdict(int))
    
    with open(file_path, "r") as file:
        for line in file:
            tweet = json.loads(line)
            # Extraemos la date del tweet y la convertimos a datetime.date
            tweet_date = datetime.strptime(tweet['date'].split("T")[0], '%Y-%m-%d').date()
            # Actualizamos el conteo de tweets para esa fecha
            date_counts[tweet_date] += 1
            # Actualizamos el conteo de tweets de usuario para esa fecha
            user_date_counts[tweet_date][tweet['user']['username']] += 1

    # Ordenamos las fechas por numero de tweets de forma descendente y se guarda el top 10
    top_dates = sorted(date_counts, key=date_counts.get, reverse=True)[:10]
    # Para cada fecha, obtenemos el usuario con mas tweets
    result = []
    for date in top_dates:
        top_user = max(user_date_counts[date], key=user_date_counts[date].get)
        result.append((date, top_user))
    
    return result


The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
peak memory: 1832.80 MiB, increment: 3.36 MiB


# Evaluacion de funciones

In [ ]:
import cProfile
import pstats

In [4]:
# Q1

# Evaluacion de uso de memoria: 

%load_ext memory_profiler
%memit q1_time("./farmers-protest-tweets-2021-2-4.json")
%memit q1_memory("./farmers-protest-tweets-2021-2-4.json")

profiler = cProfile.Profile()
profiler.enable()

# Codigo a evaluar:
q1_memory("./farmers-protest-tweets-2021-2-4.json")
q1_time("./farmers-protest-tweets-2021-2-4.json")
profiler.disable()
profiler.dump_stats("output.pstats")
stats = pstats.Stats("output.pstats")
stats.sort_stats("cumulative")
stats.print_stats(10)

Exception ignored When destroying _lsprof profiler:
Traceback (most recent call last):
  File "C:\Users\migue\AppData\Local\Temp\ipykernel_16872\1247291909.py", line 5, in <module>
RuntimeError: Cannot install a profile function while another profile function is being installed


Fri Sep 29 00:13:11 2023    output.pstats

         5202967 function calls (5202032 primitive calls) in 18.690 seconds

   Ordered by: cumulative time
   List reduced from 1036 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000   19.185    6.395 C:\Users\migue\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3472(run_code)
        3    0.000    0.000   19.185    6.395 {built-in method builtins.exec}
        1    0.015    0.015   11.772   11.772 C:\Users\migue\AppData\Local\Temp\ipykernel_16872\2600883422.py:8(q1_time)
        1    0.046    0.046    7.049    7.049 c:\Users\migue\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\json\_json.py:486(read_json)
        1    1.191    1.191    6.918    6.918 C:\Users\migue\AppData\Local\Temp\ipykernel_16872\2076002235.py:6(q1_memory)
        1    0.017    0.017    6.594    6.594 c:\Users\migue\AppData\Local\Programs\